In [16]:
import pandas as pd
import nltk
datacorp = pd.read_pickle('pickles/lhv2016t2021.pkl')
datacorp['document'] = datacorp.index

query_G = 'Audit and control, Board structure, Remuneration, Shareholder rights, Transparency and Performance'
query_S = 'Access to medicines, HIV, AIDs, Nutrition, Product safety, Community relations, Privacy and free expression, Security, Weak, governance zones, Diversity, Health and safety, ILO core conventions, Supply chain labor standards, Bribery and corruption, Political influence, Responsible marketing, Whistle-blowing systems, disclosure and reporting, Governance of sustainability issues, Stakeholder engagement, UNGC compliance'
query_E = 'Biofuels, Climate ,Emissions ,land, Biodiversity, Water, Environmental, standards, Pollution, Supply, Waste, recycling'

testdoc = datacorp.loc[datacorp['document']== '2016_ar_en_eur_con_00.txt']


In [17]:
#andmetest võtan andmepealkirjad ja tekstid nimetan dokumentideks

titles = [item for item in testdoc['document']]
documents = [item for item in testdoc['text']]


print(f'{len(documents)} documents')

1 documents


In [21]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)
def preprocess(doc):
    doc = doc.lower().split()
    stop_words = stopwords.words('english')
    doc = [w for w in doc if w not in stop_words]
    return doc

In [24]:
import re 
import string
def preprocess_query(doc):
    doc = doc.lower().split()
    doc = [remove_punc(i) for i in doc]
    stop_words = stopwords.words('english')
    doc = [w for w in doc if w not in stop_words]
    return doc

In [25]:
def remove_punc(string):
    punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''
    for ele in string:  
        if ele in punc:  
            string = string.replace(ele, "") 
    return string

In [34]:

corpus_test = preprocess(documents[0])
query_test = preprocess_query(query_S)
query_test

['access',
 'medicines',
 'hiv',
 'aids',
 'nutrition',
 'product',
 'safety',
 'community',
 'relations',
 'privacy',
 'free',
 'expression',
 'security',
 'weak',
 'governance',
 'zones',
 'diversity',
 'health',
 'safety',
 'ilo',
 'core',
 'conventions',
 'supply',
 'chain',
 'labor',
 'standards',
 'bribery',
 'corruption',
 'political',
 'influence',
 'responsible',
 'marketing',
 'whistleblowing',
 'systems',
 'disclosure',
 'reporting',
 'governance',
 'sustainability',
 'issues',
 'stakeholder',
 'engagement',
 'ungc',
 'compliance']

In [35]:
import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.models import WordEmbeddingSimilarityIndex 
corpus_t = [query_test, corpus_test]
dictionary_t = Dictionary(corpus_t)
tfidf = TfidfModel(dictionary=dictionary_t)

query_test_tf = tfidf[dictionary_t.doc2bow(query_test)]
corpus_test_tf = tfidf[dictionary_t.doc2bow(corpus_test)]
glove = api.load("glove-wiki-gigaword-50")
similarity_index_test = WordEmbeddingSimilarityIndex(glove)

similarity_matrix_test = SparseTermSimilarityMatrix(similarity_index_test, dictionary_t, tfidf)

interpret_soft_cosine_measure(query_test_tf,corpus_test_tf,dictionary_t,similarity_matrix_test)

0.02 = 0.00 (hiv:risk) + 0.00 (aids:risk) + 0.00 (labor:non) + 0.00 (engagement:activities) + 0.00 (labor:current) + 0.00 (labor:term) + 0.00 (labor:tax) + 0.00 (corruption:internal) + 0.00 (conventions:activities) + 0.00 (labor:pension) + 0.00 (labor:economic) + 0.00 (labor:consumer) + 0.00 (labor:finance) + 0.00 (labor:increase) + 0.00 (labor:changes) + 0.00 (zones:operations) + 0.00 (privacy:legal) + 0.00 (labor:change) + 0.00 (corruption:legal) + 0.00 (labor:legal) + 0.00 (labor:parties) + 0.00 (labor:commission) + 0.00 (zones:within) + 0.00 (labor:policies) + 0.00 (zones:control) + 0.00 (conventions:provisions) + 0.00 (privacy:provisions) + 0.00 (labor:policy) + 0.00 (conventions:accordance) + 0.00 (hiv:impaired) + 0.00 (medicines:products) + 0.00 (expression:recognition) + 0.00 (expression:values) + 0.00 (medicines:use) + 0.00 (conventions:principles) + 0.00 (labor:social) + 0.00 (labor:demand) + 0.00 (corruption:policies) + 0.00 (conventions:act) + 0.00 (expression:therefore) + 

In [31]:
def interpret_soft_cosine_measure(doc1, doc2, dictionary, similarity_matrix):
    word_pair_importances = dict()
    for word1_id, word1_weight in doc1:
        word1 = dictionary.id2token[word1_id]
        for word2_id, word2_weight in doc2:
            word_similarity = similarity_matrix.matrix[word1_id, word2_id]
            word_pair_importance = word1_weight * word_similarity * word2_weight
            if word_pair_importance == 0:
                continue
            word1 = dictionary.id2token[word1_id]
            word2 = dictionary.id2token[word2_id]
            if (word1, word2) not in word_pair_importances:
                word_pair_importances[word1, word2] = 0.0
            word_pair_importances[word1, word2] += word_pair_importance
    norm = 1.0
    norm *= similarity_matrix.inner_product(doc1, doc1) or 1.0
    norm *= similarity_matrix.inner_product(doc2, doc2) or 1.0
    normalized_word_pair_importances = {
        (word1, word2): word_pair_importance / norm
        for (word1, word2), word_pair_importance
        in word_pair_importances.items()
    }
    similarity = sum(normalized_word_pair_importances.values())
    normalized_word_pair_importances = sorted(normalized_word_pair_importances.items(), key=lambda x: x[1], reverse=True)
    normalized_word_pair_importances = ' + '.join(
        '{:.02f} ({}:{})'.format(word_pair_importance, word1, word2) if word1 != word2 else '{:.02f} ({})'.format(word_pair_importance, word1)
        for (word1, word2), word_pair_importance
        in normalized_word_pair_importances
    )
    print('{:.02f} = {}'.format(similarity, normalized_word_pair_importances))

In [36]:
sentence_president='Audit and control, Board structure, Remuneration, Shareholder rights, Transparency and Performance'

testdoc = datacorp.loc[datacorp['document']== '2016_ar_en_eur_con_00.txt']
document_orange = testdoc["text"][0]

s = preprocess(sentence_president)
t = preprocess(document_orange)
sentence_president2 = dictionary.doc2bow(s)

sentence_orange2 = dictionary.doc2bow(t)
similarity = similarity_matrix.inner_product(sentence_president2, sentence_orange2, normalized=True)
print('similarity = %.4f' % similarity)

NameError: name 'dictionary' is not defined